In [1]:
from IPython.display import display, Image
import seaborn as sns; sns.set()
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
%matplotlib inline
sns.set()

/jet/var/python/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def data_process(avarage_months=True, fill_na='zero'):
    #fill_na= 'zero', 'avg'
    X = pd.read_csv('BigDataSchool_features.csv')
    Y_train = pd.read_csv('BigDataSchool_train_set.csv')
    Y_test = pd.read_csv('BigDataSchool_test_set.csv')
    
    if fill_na=='zero':
        X = X.fillna(0)

    #del_list =['F23','MONTH_NUM_FROM_EVENT', 'F4', 'F5',  'F19', 'F22', 'F24', 'F31', 'F45']
    del_list =['F23','MONTH_NUM_FROM_EVENT']
    
    known_id = list(Y_train.ID)
    Uknown_id = list(Y_test.ID)
    X_train_raw = X[X.ID.isin(known_id)]
    X_test_raw = X[X.ID.isin(Uknown_id)]
    

    if avarage_months==True: # if mean thru all months is needed
        X_train_raw = X_train_raw.groupby(['ID']).mean()
        X_test_raw = X_test_raw.groupby(['ID']).mean()

    X_train = X_train_raw.drop(del_list, axis = 1)
    X_test = X_test_raw.drop(del_list, axis = 1)
    
    return X_train, X_test, Y_train, Y_test  

In [11]:
X_train, X_test, Y_train, Y_test = data_process(avarage_months=True, fill_na='zero')

In [12]:
# Use if there  avarage_months=True
X = np.array(X_train)
labels_train = (np.arange(2) == Y_train.TARGET.values[:,None]).astype(np.float32)
labels_test = (np.arange(2) == Y_test.TARGET.values[:,None]).astype(np.float32)

In [13]:
X.shape

(12890, 45)

In [14]:
X

array([[4.00000000e+00, 1.30500000e+02, 2.31666667e+01, ...,
        0.00000000e+00, 2.00000000e+00, 6.85500000e+02],
       [5.00000000e+00, 1.36500000e+02, 1.91666667e+01, ...,
        0.00000000e+00, 1.16666667e+00, 3.86866667e+03],
       [2.00000000e+01, 1.65000000e+01, 2.33333333e+01, ...,
        0.00000000e+00, 1.00000000e+00, 1.69833333e+02],
       ...,
       [1.60000000e+01, 4.55000000e+01, 2.70000000e+01, ...,
        8.33333333e-01, 1.33333333e+00, 7.51016667e+03],
       [2.00000000e+00, 9.88333333e+01, 2.00000000e+01, ...,
        0.00000000e+00, 1.00000000e+00, 9.15000000e+02],
       [1.00000000e+01, 9.96666667e+01, 2.85000000e+01, ...,
        0.00000000e+00, 1.00000000e+00, 1.73783333e+03]])

In [15]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, labels_train ,test_size=0.3, random_state=32)

In [16]:
element_size = 45 #  dimension of each vector in our sequence
time_steps = 1   #  number of such elements in a sequence
num_classes = 2
batch_size = 128
hidden_layer_size = 256 # neurons number
n_epochs = 10000
learning_rate = 0.01

graph = tf.Graph()
with graph.as_default():
    _inputs = tf.placeholder(tf.float32,shape=[None, time_steps, element_size])
    y = tf.placeholder(tf.float32, shape=[None,num_classes])
    # TensorFlow built-in functions
    rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_layer_size)
    outputs, states = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)   # outputs a single tensor for all outputs at every time step
    Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
    mean=0,stddev=.01))
    bl = tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))
    def get_linear_layer(vector):
        return tf.matmul(vector, Wl) + bl
    last_rnn_output = outputs[:,-1,:]
    final_output = get_linear_layer(last_rnn_output)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,
    labels=y)
    cross_entropy = tf.reduce_mean(softmax)
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(softmax)
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(final_output,1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    
    def final_predicion(x_test, clf, save_csv = False):
        y_pred = clf.predict(x_test)
        if save_csv == True:
            pd.DataFrame({'ID': Y_test.ID, 'TARGET': y_pred}).to_csv('submit.csv', index=False)
            return 'Submittal dataset is saved as submit.csv'
        return pd.Series(y_pred, index=Y_test.ID)
    # data for model saving
    #saver = tf.train.Saver(tf.all_variables())
    path = os.getcwd()+'/saved_model/my_model'

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [20]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    test_dataset = test_dataset.reshape((-1, time_steps, element_size))
    accur = 0
    
    for i in range(10000):
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        batch_x, batch_y = train_dataset[offset:(offset+batch_size),:], train_labels[offset:(offset+batch_size)]
        batch_x = batch_x.reshape((batch_size, time_steps, element_size))
        #acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
        #loss = sess.run(cross_entropy,feed_dict={_inputs:batch_x, y:batch_y})
        sess.run(train_step,feed_dict={_inputs:batch_x, y:batch_y})
        if i % 1000 == 0:
            acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
            loss = sess.run(cross_entropy,feed_dict={_inputs:batch_x, y:batch_y})
            
            y_test = batch_y
            y_pred = sess.run(tf.nn.softmax(final_output), feed_dict={_inputs: batch_x, y: batch_y})
            auc_roc = roc_auc_score(y_test, y_pred)
            print ("Iter " + str(i) + ", Minibatch Loss= " + \
                "{:.6f}".format(loss) + ", Training Accuracy= " + \
                    "{:.5f}".format(acc) + ", Training AUC ROC= " + \
                    "{:.5f}".format(auc_roc))
            # Testing accuracy and AUR ROC
            y_test = test_labels
            y_pred = sess.run(tf.nn.softmax(final_output), feed_dict={_inputs: test_dataset, y: test_labels})
            auc_roc = roc_auc_score(y_test, y_pred)            
            print ("Testing Accuracy:",sess.run(accuracy, feed_dict={_inputs: test_dataset, y: test_labels}),\
                  ", Testing AUC ROC:","{:.5f}".format(auc_roc))
            # save the model with best auc_roc evaluatiuon
            if auc_roc > accur:
                saver = tf.train.Saver(tf.all_variables())
                print('auc_roc=',auc_roc,'is greater than acc=',accur) # print just to appoint that MAX AUC_ROC is changed
                saver.save(sess, path)
                accur = auc_roc

Initialized
Iter 0, Minibatch Loss= 0.412069, Training Accuracy= 90.62500, Training AUC ROC= 0.66739
Testing Accuracy: 91.69899 , Testing AUC ROC: 0.49501
auc_roc= 0.495009771940923 is greater than acc= 0
Iter 1000, Minibatch Loss= 0.223245, Training Accuracy= 91.40625, Training AUC ROC= 0.88500
Testing Accuracy: 91.67313 , Testing AUC ROC: 0.67284
auc_roc= 0.67283678738165 is greater than acc= 0.495009771940923
Iter 2000, Minibatch Loss= 0.282136, Training Accuracy= 89.06250, Training AUC ROC= 0.79073
Testing Accuracy: 91.69899 , Testing AUC ROC: 0.64221
Iter 3000, Minibatch Loss= 0.314224, Training Accuracy= 88.28125, Training AUC ROC= 0.77068
Testing Accuracy: 91.466255 , Testing AUC ROC: 0.63001
Iter 4000, Minibatch Loss= 0.131795, Training Accuracy= 96.09375, Training AUC ROC= 0.80325
Testing Accuracy: 91.724846 , Testing AUC ROC: 0.64478
Iter 5000, Minibatch Loss= 0.217488, Training Accuracy= 93.75000, Training AUC ROC= 0.75625
Testing Accuracy: 91.59555 , Testing AUC ROC: 0.6438

In [21]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    #saver = tf.train.Saver()
    saver.restore(sess, path)
    print("Model restored.")
    print('Initialized')
    y_test = test_labels
    y_pred = sess.run(tf.nn.softmax(final_output), feed_dict={_inputs: test_dataset, y: test_labels})
    auc_roc = roc_auc_score(y_test, y_pred) 
    print ("Testing Accuracy:",sess.run(accuracy, feed_dict={_inputs: test_dataset, y: test_labels}),\
                  ", Testing AUC ROC:","{:.5f}".format(auc_roc))
    print(sess.run(bl))

INFO:tensorflow:Restoring parameters from /jet/prs/workspace/kyivstar2018/saved_model/my_model
Model restored.
Initialized
Testing Accuracy: 91.54384 , Testing AUC ROC: 0.67285
[-0.16812158  0.172959  ]
